<div align="center">
<h1 style="color:#1565c0; font-size:2.8em; font-weight:bold; margin-bottom:0.2em; margin-top:0.5em;">Практична робота №4</h1>
<h3 style="color:#1976d2; margin-top:0; font-weight:normal;">Кластеризацiя засобами
бiблiотеки Scikit-Learn Python <b>Scikit-Learn Python</b></h3>
<br>
<br>
<b>Варіант 12</b><br>
<b>Автори:</b><br>
Плахтій Злата, КА-32<br>
Пастушенко Максим, КА-33<br>
Романов Фелікс, КА-33
</div>


<b>Хід роботи:</b>
1.  Представити початковi данi графiчно.
2.  Побудувати модель кластеризацiї згiдно з варiантом.
3.  Виконати кластеризацiю даних на основi моделi.
4.  Представити розбиття на кластери графiчно (наприклад, рiзними кольорами).
5.  Розрахувати час кластеризацiї. Оцiнити швидкодiю методу на надвеликих наборах даних (наприклад, при збiльшеннi кiлькостi точок до 100 000 i бiльше).
6.  Побудувати кiлька альтернативних моделей:
    * шляхом змiни значень параметрiв основної моделi;
    * використати рiзнi функцiї вiдстанi (де це можливо);
    * задати рiзнi значення кiлькостi кластерiв (де це параметр).
7.  Для кожної альтернативної моделi розрахувати метрики якостi кластеризацiї з `sklearn.metrics` (тiльки метрики згiдно з варiантом):
    * Estimated Number of Clusters
    * Adjusted Rand Index
    * Adjusted Mutual Information
    * Homogeneity
    * Completeness
    * V-measure
    * Silhouette Coefficient
    * Calinski-Harabasz Index
    * Davies-Bouldin index
    * Contingency Matrix
8.  Виконати аналiз результатiв одним з неформальних методiв (тiльки методом згiдно з варiантом):
    * чи є розбиття стабiльним на пiдвибiрках даних?
    * чи є розбиття стабiльним пiсля видалення окремих об’єктiв?
    * чи є розбиття стабiльним пiсля змiни порядку об’єктiв?
    * чи iснує взаємозв’язок мiж результатами i змiнними, якi не враховувалися?
    * чи можна iнтерпретувати результати?
9.  Виконати пункти 1-8 для заданих двох наборiв даних рiзної форми.
10. Зробити висновки про якiсть роботи моделей та про швидкодiю методу.
11. Оцiнити результати (на основi метрик та неформальних методiв). Спробувати пiдiбрати найкращу модель для кожного набору даних
## Завдання (Варіант 12)
* **Алгоритм:** `Birch`
* **Метрики якостi:**
    * Estimated number of clusters
    * Adjusted Rand Index
    * Adjusted Mutual Information
    * Silhouette Coefficient
* **Неформальний метод:**
    * Чи є розбиття стабiльним пiсля змiни порядку об’єктiв у множинi об’єктiв?
* **Початковi данi:**
    1.  `sklearn.datasets.make_moons`
    2.  `sklearn.datasets.load_iris`

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Налаштування стилю графіків
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


# ============================================================================
# КРОК 1: ЗАВАНТАЖЕННЯ ТА ПЕРВИННИЙ АНАЛІЗ ДАНИХ
# ============================================================================

print("\n" + "="*70)
print("КРОК 1: ЗАВАНТАЖЕННЯ ТА ПЕРВИННИЙ АНАЛІЗ ДАНИХ")
print("="*70)

# Визначення шляху до файлу
script_directory = Path(__file__).parent
iris_filepath = script_directory / "data" / "iris_dataset.csv"

# Завантаження даних
print(f"\n📁 Завантаження даних з: {iris_filepath}")
df = pd.read_csv(iris_filepath)

print("\n✅ Дані успішно завантажено!")

# 1.1 Базова інформація про датасет
print("\n" + "-"*70)
print("1.1 БАЗОВА ІНФОРМАЦІЯ ПРО ДАТАСЕТ")
print("-"*70)

print(f"\n📊 Розмірність датасету: {df.shape}")
print(f"   - Кількість зразків: {df.shape[0]}")
print(f"   - Кількість ознак: {df.shape[1] - 1} (без target)")

print("\n📋 Перші 5 рядків:")
print(df.head())

print("\n📋 Останні 5 рядків:")
print(df.tail())

print("\n🏷️ Типи даних:")
print(df.dtypes)

print("\n🔢 Інформація про датасет:")
df.info()

# 1.2 Статистичні характеристики
print("\n" + "-"*70)
print("1.2 СТАТИСТИЧНІ ХАРАКТЕРИСТИКИ")
print("-"*70)

print("\n📈 Описова статистика для всіх ознак:")
print(df.describe())

print("\n📊 Статистика по цільовій змінній (target):")
print(df['target'].value_counts().sort_index())

target_names = ['Setosa (0)', 'Versicolor (1)', 'Virginica (2)']
print("\nРозподіл класів:")
for i, name in enumerate(target_names):
    count = (df['target'] == i).sum()
    percentage = (count / len(df)) * 100
    print(f"  {name}: {count} зразків ({percentage:.1f}%)")

# 1.3 Перевірка на пропущені значення
print("\n" + "-"*70)
print("1.3 ПЕРЕВІРКА НА ПРОПУЩЕНІ ЗНАЧЕННЯ")
print("-"*70)

missing_values = df.isnull().sum()
print("\n🔍 Кількість пропущених значень по стовпцях:")
print(missing_values)

if missing_values.sum() == 0:
    print("\n✅ Пропущені значення відсутні!")
else:
    print(f"\n⚠️ Загальна кількість пропущених значень: {missing_values.sum()}")

# 1.4 Кореляційна матриця
print("\n" + "-"*70)
print("1.4 КОРЕЛЯЦІЙНА МАТРИЦЯ МІЖ ОЗНАКАМИ")
print("-"*70)

# Відокремлюємо ознаки (без target)
feature_columns = [col for col in df.columns if col != 'target']
X = df[feature_columns].values
y = df['target'].values

correlation_matrix = df[feature_columns].corr()
print("\n📊 Кореляція між ознаками:")
print(correlation_matrix)

# ============================================================================
# КРОК 2: ВІЗУАЛІЗАЦІЯ ПОЧАТКОВИХ ДАНИХ
# ============================================================================

print("\n" + "="*70)
print("КРОК 2: ВІЗУАЛІЗАЦІЯ ПОЧАТКОВИХ ДАНИХ")
print("="*70)

# 2.1 Кореляційна матриця (heatmap)
print("\n📊 Створення теплової карти кореляцій...")
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Кореляційна матриця ознак датасету Iris', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(script_directory / 'iris_01_correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Збережено: iris_01_correlation_matrix.png")

# 2.2 Box plots для кожної ознаки
print("\n📊 Створення box plots для кожної ознаки...")
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, col in enumerate(feature_columns):
    axes[idx].boxplot([df[df['target'] == i][col] for i in range(3)],
                      labels=target_names)
    axes[idx].set_title(f'Box Plot: {col}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Значення')
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Розподіл ознак по класах (Box Plots)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(script_directory / 'iris_02_boxplots.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Збережено: iris_02_boxplots.png")

# 2.3 Pair plot (матриця діаграм розсіювання)
print("\n📊 Створення pair plot (може зайняти кілька секунд)...")
pairplot_df = df.copy()
pairplot_df['species'] = pairplot_df['target'].map({0: 'Setosa', 1: 'Versicolor', 2: 'Virginica'})

g = sns.pairplot(pairplot_df, hue='species', vars=feature_columns,
                 diag_kind='kde', plot_kws={'alpha': 0.6, 's': 50, 'edgecolor': 'k'},
                 height=2.5)
g.fig.suptitle('Pair Plot: Всі пари ознак датасету Iris', y=1.02, fontsize=14, fontweight='bold')
plt.savefig(script_directory / 'iris_03_pairplot.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Збережено: iris_03_pairplot.png")

# 2.4 Scatter plots для кожної пари ознак з реальними мітками
print("\n📊 Створення scatter plots для всіх пар ознак...")
from itertools import combinations

feature_pairs = list(combinations(range(len(feature_columns)), 2))
n_pairs = len(feature_pairs)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

colors = ['red', 'green', 'blue']
markers = ['o', 's', '^']

for idx, (i, j) in enumerate(feature_pairs):
    for class_idx in range(3):
        mask = y == class_idx
        axes[idx].scatter(X[mask, i], X[mask, j], 
                         c=colors[class_idx], 
                         marker=markers[class_idx],
                         label=target_names[class_idx],
                         alpha=0.6, 
                         edgecolors='k',
                         s=50)
    
    axes[idx].set_xlabel(feature_columns[i], fontsize=10)
    axes[idx].set_ylabel(feature_columns[j], fontsize=10)
    axes[idx].set_title(f'{feature_columns[i]} vs {feature_columns[j]}', 
                       fontsize=11, fontweight='bold')
    axes[idx].legend(fontsize=8)
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Scatter Plots: Всі пари ознак з реальними класами', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(script_directory / 'iris_04_scatter_pairs.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Збережено: iris_04_scatter_pairs.png")

# 2.5 PCA для зменшення розмірності та візуалізації
print("\n📊 Виконання PCA для візуалізації в 2D...")

# Стандартизація даних (важливо для PCA!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA з 2 компонентами
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

print(f"\n📉 Пояснена дисперсія (2 компоненти):")
print(f"   - PC1: {pca_2d.explained_variance_ratio_[0]:.4f} ({pca_2d.explained_variance_ratio_[0]*100:.2f}%)")
print(f"   - PC2: {pca_2d.explained_variance_ratio_[1]:.4f} ({pca_2d.explained_variance_ratio_[1]*100:.2f}%)")
print(f"   - Загалом: {sum(pca_2d.explained_variance_ratio_):.4f} ({sum(pca_2d.explained_variance_ratio_)*100:.2f}%)")

# Візуалізація PCA 2D
plt.figure(figsize=(12, 8))
for class_idx in range(3):
    mask = y == class_idx
    plt.scatter(X_pca_2d[mask, 0], X_pca_2d[mask, 1],
               c=colors[class_idx],
               marker=markers[class_idx],
               label=target_names[class_idx],
               alpha=0.6,
               edgecolors='k',
               s=100)

plt.xlabel(f'Перша головна компонента (PC1) - {pca_2d.explained_variance_ratio_[0]*100:.2f}%', 
           fontsize=12)
plt.ylabel(f'Друга головна компонента (PC2) - {pca_2d.explained_variance_ratio_[1]*100:.2f}%', 
           fontsize=12)
plt.title('PCA: Датасет Iris у 2D просторі (з реальними класами)', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(script_directory / 'iris_05_pca_2d.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Збережено: iris_05_pca_2d.png")

# 2.6 PCA з 3 компонентами для 3D візуалізації
print("\n📊 Виконання PCA для візуалізації в 3D...")

pca_3d = PCA(n_components=3)
X_pca_3d = pca_3d.fit_transform(X_scaled)

print(f"\n📉 Пояснена дисперсія (3 компоненти):")
for i in range(3):
    print(f"   - PC{i+1}: {pca_3d.explained_variance_ratio_[i]:.4f} ({pca_3d.explained_variance_ratio_[i]*100:.2f}%)")
print(f"   - Загалом: {sum(pca_3d.explained_variance_ratio_):.4f} ({sum(pca_3d.explained_variance_ratio_)*100:.2f}%)")

# 3D Scatter Plot
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

for class_idx in range(3):
    mask = y == class_idx
    ax.scatter(X_pca_3d[mask, 0], X_pca_3d[mask, 1], X_pca_3d[mask, 2],
              c=colors[class_idx],
              marker=markers[class_idx],
              label=target_names[class_idx],
              alpha=0.6,
              edgecolors='k',
              s=50)

ax.set_xlabel(f'PC1 ({pca_3d.explained_variance_ratio_[0]*100:.1f}%)', fontsize=11)
ax.set_ylabel(f'PC2 ({pca_3d.explained_variance_ratio_[1]*100:.1f}%)', fontsize=11)
ax.set_zlabel(f'PC3 ({pca_3d.explained_variance_ratio_[2]*100:.1f}%)', fontsize=11)
ax.set_title('PCA: Датасет Iris у 3D просторі (з реальними класами)', 
            fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
plt.tight_layout()
plt.savefig(script_directory / 'iris_06_pca_3d.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Збережено: iris_06_pca_3d.png")

# 2.7 Розподіл кожної ознаки (гістограми)
print("\n📊 Створення гістограм розподілу ознак...")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, col in enumerate(feature_columns):
    for class_idx in range(3):
        mask = y == class_idx
        axes[idx].hist(X[mask, idx], bins=20, alpha=0.5, 
                      label=target_names[class_idx], 
                      color=colors[class_idx],
                      edgecolor='black')
    
    axes[idx].set_xlabel(col, fontsize=10)
    axes[idx].set_ylabel('Частота', fontsize=10)
    axes[idx].set_title(f'Розподіл: {col}', fontsize=11, fontweight='bold')
    axes[idx].legend(fontsize=8)
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Гістограми розподілу ознак по класах', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(script_directory / 'iris_07_histograms.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Збережено: iris_07_histograms.png")

# ============================================================================
# ЗБЕРЕЖЕННЯ ОБРОБЛЕНИХ ДАНИХ
# ============================================================================

print("\n" + "="*70)
print("ЗБЕРЕЖЕННЯ ОБРОБЛЕНИХ ДАНИХ")
print("="*70)

# Збереження стандартизованих даних для подальшого використання
scaled_df = pd.DataFrame(X_scaled, columns=feature_columns)
scaled_df['target'] = y
scaled_filepath = script_directory / "data" / "iris_scaled.csv"
scaled_df.to_csv(scaled_filepath, index=False)
print(f"\n💾 Стандартизовані дані збережено: {scaled_filepath}")

# Збереження PCA даних
pca_df = pd.DataFrame(X_pca_2d, columns=['PC1', 'PC2'])
pca_df['target'] = y
pca_filepath = script_directory / "data" / "iris_pca.csv"
pca_df.to_csv(pca_filepath, index=False)
print(f"💾 PCA дані (2D) збережено: {pca_filepath}")

print("\n" + "="*70)
print("✅ КРОКИ 1-2 ЗАВЕРШЕНО УСПІШНО!")
print("="*70)
print("\n📊 Створено 7 графіків:")
print("   1. iris_01_correlation_matrix.png - Кореляційна матриця")
print("   2. iris_02_boxplots.png - Box plots")
print("   3. iris_03_pairplot.png - Pair plot")
print("   4. iris_04_scatter_pairs.png - Scatter plots всіх пар")
print("   5. iris_05_pca_2d.png - PCA 2D візуалізація")
print("   6. iris_06_pca_3d.png - PCA 3D візуалізація")
print("   7. iris_07_histograms.png - Гістограми розподілу")

print("\n💡 Наступний крок: Побудова базової моделі BIRCH")
print("="*70)


КРОК 1: ЗАВАНТАЖЕННЯ ТА ПЕРВИННИЙ АНАЛІЗ ДАНИХ


NameError: name '__file__' is not defined